## Block Matching Regisrtation

This notebook outputs transformaed pointclouds + alignmnet matrices + warped images. 
This notebut is run in WSL since it is using ants , which is not available for windows. 

For vizualisation, go to " NC_Control_View" notebook , which is run from windows ... 

In [1]:
import pwreg as pw

import pickle
import pandas as pd

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/home/ply/anaconda3/envs/ants/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
project_path = "/mnt/d/Code/repos/UGPy"

### Global image alignment 
( TODO : allow rigid ) 

In [3]:
mask_tp2 = None
mask_tp1 = None

In [4]:
spec = {
    'fixed':
        {'filename' : f'{project_path}/data/changes/learners/1-23C4/crops/1-23E2_left_main.tif',
         'resolution' : [0.658, 0.23, 0.23],
         'mask' : mask_tp1,
         'img' : None, 
         'info' : None},
    'moving':
        {'filename' : f'{project_path}/data/changes/learners/1-23C4/crops/1-23E4_left_main.tif',
         'resolution' : [0.658, 0.23, 0.23],
         'mask' : mask_tp2,
         'img' : None, 
         'info' : None}
        }

imp = pw.ImagePair.from_dict(spec)
imp.shape

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


{'fixed': (121, 799, 420), 'moving': (131, 799, 420)}

In [5]:
# put the whole image size here
transform_folder = f"{project_path}/tmp"
imp.register(transform_folder, fixed_size=[121, 799, 420], moving_size=[131, 799, 420], fixed_overlap=0, moving_overlap=0, verbose = True)


  0%|                                                                                             | 0/1 [00:00<?, ?it/s]

fixed : ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (121, 799, 420)
	 Spacing    : (0.658, 0.23, 0.23)
	 Origin     : (0.0, 0.0, 0.0)
	 Direction  : [1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0]

moving : ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (131, 799, 420)
	 Spacing    : (0.658, 0.23, 0.23)
	 Origin     : (0.0, 0.0, 0.0)
	 Direction  : [1.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0]



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:46<00:00, 106.19s/it]


In [9]:
imp.save(f'{project_path}/data/changes/learners/1-23C4/registered/crops/pw_reg/global', padding = 0, info = 'Global alignment, after cutting ')

### Block affine registration

In [6]:
# use the pre-registered 
imp2 = pw.ImagePair(imp.fixed, imp.alignment[0].warp(interpolator = 'linear'))
imp2.shape

{'fixed': (121, 799, 420), 'moving': (121, 799, 420)}

In [12]:
# put the whole image size here
block_size = [20,200,200]
overlap = [15,150,150]
imp2.register(transform_folder, fixed_size=block_size, moving_size=block_size, fixed_overlap=overlap, moving_overlap=overlap, verbose = False)

/mnt/d/Code/repos/pwreg/src/pwreg/core.py:164: UserWarning:

Specified voxel size + overlap don't cover the whole image.Image size is [121 799 420], block size [ 20 200 200], overlap [ 15 150 150] results in [21.2 13.0 5.4] blocks.
Leaving some image out. 

100%|███████████████████████████████████████████████████████████████████████████████| 1260/1260 [51:21<00:00,  2.45s/it]


Let's try with the overlap:

In [15]:
imp2.save(f'{project_path}/data/changes/learners/1-23C4/registered/crops/pw_reg/pw_after_glob_4', padding = [3,10,10],
          info = 'Pw registration after global;\
                  fixed= changes/learners/1-23C4/crops/1-23E2_left_main.tif, moving = data/changes/learners/1-23C4/crops/1-23E4_left_main.tif;\
                  block_size = [20,200,200],overlap = [15,150,150]')

### Apply Block Affine to probability maps

In [19]:
fixed = pw.Image([0.658, 0.23, 0.23],
                 filename = f'{project_path}/data/changes/learners/1-23C4/prob_map/crops/map_id_rzn02jyz_ckpt_3m51q4ne_1-23E2_left_main.tif',
                 mask = mask_tp1) # tp 1
moving = pw.Image([0.658, 0.23, 0.23], 
                  filename = f'{project_path}/data/changes/learners/1-23C4/prob_map/crops/map_id_74tmf0l2_ckpt_3m51q4ne_1-23E4_left_main.tif',
                  mask = mask_tp2) # tp 2
# put the whole image size here
voxels_f = fixed.split([121, 799, 420],0)
voxels_m = moving.split([131, 799, 420],0)

In [20]:
pair_pm = pw.BlockPair(voxels_f[0],voxels_m[0], alignment=imp.alignment[0].alignment)
pair_pm.warp().imwrite(
    f'{project_path}/data/changes/learners/1-23C4/registered/crops/pw_reg/pw_after_glob_4/global_map_id_74tmf0l2_1-23E4_to_1-23E2_left_main.tif')

In [21]:
warped = pair_pm.warp()

In [22]:
voxels_f = fixed.split(block_size,overlap)
voxels_m = warped.split(block_size,overlap)

In [24]:
pairs = [pw.BlockPair(voxf,voxm) for voxf,voxm in zip(voxels_f,voxels_m)]
for pair, imp2_pair in zip(pairs,imp2.alignment): 
    pair.alignment = imp2_pair.alignment

In [26]:
padding = [3,10,10]
folder = f'{project_path}/data/changes/learners/1-23C4/registered/crops/pw_reg/pw_after_glob_4'

# fixed image
blc1 = [pair.blc1 for pair in pairs]
bv1 = pw.BlockView(blc1, padding)
bv1.write_volume(f"{folder}/pmap_fixed.tif")

# moving image
blc2 = [pair.blc2 for pair in pairs]
bv2 = pw.BlockView(blc2, padding)
bv2.write_volume(f"{folder}/pmap_moving.tif")

# warped (registered) image
bp = pw.BlockPairView(pairs, padding)
bp.write_volume(f"{folder}/pmap_warped.tif")

### Compare probability maps

In [45]:
import numpy as np
import tifffile as tif
import copy

In [36]:
prob1_view = bv1.zero_volume()
prob2_view = bv1.zero_volume()

In [42]:
SCALE = 1000
for map_pair, img_pair in zip(pairs,imp2.alignment): 
    
    # create mask based on where images are nonzero
    mask = np.logical_and((img_pair.blc1.crop() > 0),(img_pair.blc2.crop() > 0))
    
    # sum up proabblities in the mask
    prob1_avg = np.mean(map_pair.blc1.crop()[mask])
    prob2_avg = np.mean(map_pair.blc2.crop()[mask])
    
    # write to pair
    block = img_pair.blc1
    z0, y0, x0 = (block.size + padding) * block.idx
    z1, y1, x1 = (block.size + padding) * block.idx + block.size
    
    prob1_view[z0:z1, y0:y1, x0:x1] = prob1_avg * SCALE
    prob2_view[z0:z1, y0:y1, x0:x1] = prob2_avg * SCALE

In [43]:
tif.imsave(f"{folder}/pmap_fixed_avg.tif", np.expand_dims(prob1_view, axis=1).astype(np.uint16), imagej=True)
tif.imsave(f"{folder}/pmap_warped_avg.tif", np.expand_dims(prob2_view, axis=1).astype(np.uint16), imagej=True)

In [46]:
diff_view = prob2_view-prob1_view

change_img_pos = copy.deepcopy(np.round(diff_view)).astype(np.int16)
change_img_neg = copy.deepcopy(np.round(diff_view)).astype(np.int16)

print(np.max(change_img_pos.flatten()))
print(np.min(change_img_pos.flatten()))   

change_img_pos[change_img_pos<0] = 0
change_img_neg[change_img_neg>0] = 0
change_img_neg = -change_img_neg

print(np.max(change_img_pos.flatten()))
print(np.min(change_img_pos.flatten())) 

print(np.max(change_img_neg.flatten()))
print(np.min(change_img_neg.flatten())) 

change_pos = pw.Image([0.658, 0.23, 0.23], img = change_img_pos)
change_neg = pw.Image([0.658, 0.23, 0.23], img = change_img_neg)
change_pos.imwrite( f"{folder}/pmap_change_up.tif")
change_neg.imwrite( f"{folder}/pmap_change_down.tif")

282
-1468
282
0
1468
0
